In [1]:
# Импортируем стандартные библиотеки
import os
import math
import time
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.dates as mdates

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)


#Импортируем инструмент масштабирования
from sklearn.preprocessing import StandardScaler

# Импортируем слои и модели НС
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.models import Sequential
from keras.models import load_model
from keras.metrics import mean_squared_error

In [2]:
# Загружаем данные
df = pd.read_csv('../data/cleaned_crypto_all_features.csv', header=[0,1], skiprows=[2], index_col=0)
df.index.name = 'time'
df.index = pd.to_datetime(df.index)
df = df[:-1].astype(float) # удаляем последнюю строку (отсутсвуют closing prices)

### Предсказание для нестационарных цен закрытия 

In [3]:
# Функция для поготовки выборок

def prepare_samples(df, window_size, train_ed, target_crypto):
    """
    Подготавливает выборки для LSTM. 
    
    Входные данные:
        df: датафрейм, который будет разделён на 2 выборки (обучающую/тестовую)
        window_size: размер каждой выборки (24 для интервалов в 1 день)
        train_ed: последняя дата для обучающей выборки
    
    Функция возвращает:
        выборки: выборки готовые для применения в LSTM (train/test)
        выходные данные: цены, которые будут предсказаны (train/test)
        mean, var: матожидание и дисперисия обучающей выборки для цены закрытия 
    """
    train_df = df.ix[:train_ed]
    
    scl = StandardScaler()
    scl.fit(train_df)
    
    scl_arr = scl.transform(df)
    mean = scl.mean_[train_df.columns.get_loc(target_crypto+'_close')]
    var = scl.var_[train_df.columns.get_loc(target_crypto+'_close')]
    
    samples = []
    outputs = []
    
    for i in range(scl_arr.shape[0]-window_size):
        temp_sample = scl_arr[i:(i+window_size),:]
        if (i+window_size) < scl_arr.shape[0]:
            temp_output = scl_arr[(i+window_size),0]
        else:
            temp_output = np.nan
        samples.append(temp_sample)
        outputs.append(temp_output)
    
    n_train = len(train_df.index)
    train_samples = np.array(samples[:n_train])
    test_samples = np.array(samples[n_train:])
    train_outputs = np.array(outputs[:n_train])
    test_outputs = np.array(outputs[n_train:])
    
    return train_samples, test_samples, train_outputs, test_outputs, scl, mean, var

In [4]:
# Функция для создания LSTM.

def build_model(inputs, n_units, activ_func="linear", dropout=0.2, loss="mae", optimizer="Nadam"):
    """
    
    Входные данные:
        inputs: матрица для подачи в data frame (# samples, # time steps, # features) 
        n_units: длина предсказания 
        
    Возвращает:
        model: LSTM (готовую для обучения)
        
    """
    model = Sequential()

    model.add(LSTM(n_units[0], return_sequences=True, input_shape=(inputs.shape[1], inputs.shape[2])))
    model.add(Dropout(dropout))
    
    model.add(LSTM(n_units[1], return_sequences=False))
    model.add(Dropout(dropout))

    model.add(Dense(units=n_units[2]))
    model.add(Activation(activ_func))

    model.compile(loss=loss, optimizer=optimizer)
    
    return model

In [5]:
# Функция для генерации предсказаний для тестовой выборки. 
# Замечание:Здесь мы используем подход "walk forward" в котором выходные данные предсказания LSTM 
# становятся входными данными для последующих временных шагов.

def pred_test_set(model, test_df, window_size, tr_mean, tr_var, wreal):
    """
    
    Входные данные:
        model: LSTM 
        test_df: data frame, содержащий все данные, необходимые для тестовой выборки 
        window_size: длина каждой выборки
        tr_mean, tr_var: параметры из обучающего набора, используемые для масштабирования цен закрытия
        wreal: логическое значение, указывающее, используются ли прогнозируемые/фактические значения при 
        выполнении прогнозов на один шаг вперед для набора тестов (True: фактическое, False: прогнозируемое)
    
    Возвращает:
        test_preds: прогнозируемые цены закрытия (перемасштабировано по среднему, дисперсии)
    
    """
    test_preds = []
    
    for i in range(len(test_df.index)-window_size):
        temp_sample = test_df.iloc[i:(i+window_size),:].values
        sample_arr = np.reshape(temp_sample, (1,window_size,len(test_df.columns)))
        prediction = model.predict(sample_arr)
        
        if wreal:
            scaled_prediction = prediction * math.sqrt(tr_var) + tr_mean
            test_preds.append(float(scaled_prediction))
        else:
            #Заменить значение в test_df прогнозом, добавить масштабированный прогноз в test_preds.
            test_df.iat[(i+window_size),0] = prediction
            scaled_prediction = prediction * math.sqrt(tr_var) + tr_mean
            test_preds.append(float(scaled_prediction))
        
    return np.array(test_preds)

In [6]:
train_end = pd.to_datetime('2017/11/30 12:00:00')
test_start = pd.to_datetime('2017/11/30 13:00:00')
crypto_str = 'ETH_USD'

sub_df = df.xs(crypto_str, level='fsym_tsym', axis=1)
sub_df.columns = [crypto_str + '_' + s for s in sub_df.columns]
xmr_df = df.xs('XMR_USD', level='fsym_tsym', axis=1)
xmr_df.columns = ['XMR_USD' + '_' + s for s in xmr_df.columns]
merge_df = sub_df.merge(xmr_df, left_index=True, right_index=True)
tr_samples, te_samples, tr_outputs, te_outputs, tr_scl, tr_m, tr_v = prepare_samples(merge_df, 12, train_end, crypto_str)
train_df = merge_df.ix[:train_end]
test_df = merge_df.ix[test_start:] 
scl_test_df = pd.DataFrame(tr_scl.transform(test_df), columns=merge_df.columns).set_index(test_df.index)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated



In [7]:
np.random.seed(42)

# Если модель уже была обучена/сохранена, загрузите ее вместо повторного обучения..
model_name = 'lstm_saved_wxmr.h5'
if os.path.isfile(model_name):
    model = load_model(model_name)
else:
    # Инициализирцем LSTM.
    model = build_model(tr_samples, [20,20,1])
    
    # Обучаем LSTM на обучающих выборках, сохраняя информацию об обучении в tr_history.
    tr_history = model.fit(tr_samples, tr_outputs, epochs=5, batch_size=1, verbose=1, shuffle=True)
    model.save('lstm_saved_wxmr.h5')

In [8]:
# Генирируем прогнозы для тренировочной выборки
tr_predictions = model.predict(tr_samples)
tr_rescaled_preds = tr_predictions * math.sqrt(tr_v) + tr_m
train_dates = df.index[12:1794]
tr_pred_df = pd.DataFrame(tr_rescaled_preds, index=train_dates)
tr_pred_df.columns = ['pred_close']

# Генирируем прогнозы для тестовой выборки
te_real_predictions = pred_test_set(model, scl_test_df, 12, tr_m, tr_v, wreal=True)
te_fcast_predictions = pred_test_set(model, scl_test_df, 12, tr_m, tr_v, wreal=False)
test_dates = df.index[-205:]
#te_rescaled_outputs = te_outputs * math.sqrt(tr_v) + tr_m
te_real_pred_df = pd.DataFrame(te_real_predictions, index=test_dates)
te_fcast_pred_df = pd.DataFrame(te_fcast_predictions, index=test_dates)
te_real_pred_df.columns = ['pred_close']

In [14]:
# Создаём траекторию.
def create_trace(df, color, label):
    dates = df.index 
    prices = df[df.columns[0]].values

    trace = go.Scatter(
        x = dates,
        y = prices,
        name = label,
        line = dict(color=color)
    )
    return trace

pred_trace = create_trace(tr_pred_df, 'red', 'Предсказанная')
act_trace = create_trace(train_df, 'blue', 'Фактическая')
data = [pred_trace, act_trace]

In [15]:
layout = dict(title = 'Цена Закрытия Ethereum (Обучающая)',
              xaxis = dict(title = 'Дата'),
              yaxis = dict(title = 'Цена закрытия'),
              )

fig = dict(data=data, layout=layout)
iplot(fig, filename='training-prices')

In [12]:
pred_trace = create_trace(te_real_pred_df, 'red', 'Предсказанная')
act_trace = create_trace(test_df, 'blue', 'Фактическая')
data = [pred_trace, act_trace]

layout = dict(title = 'Цена Закрытия Ethereum (Тестовая)',
              xaxis = dict(title = 'Дата'),
              yaxis = dict(title = 'Цена закрытия'),
              )

fig = dict(data=data, layout=layout)
iplot(fig, filename='test-prices')